# **Notebook approche non supervisée**

## **Installation des module et des packtages**

In [1]:
!pip install --upgrade pyLDAvis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from wordcloud import WordCloud
from PIL import Image
from sklearn.decomposition import LatentDirichletAllocation
from time import time
import pyLDAvis
import pyLDAvis.gensim
from sklearn.metrics import f1_score, accuracy_score, jaccard_score, precision_score, recall_score

In [3]:
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_

True

## **Importation des données**

In [4]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Projet 5/my_data.csv")
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount
0,minification failed returning unminified content,made first website using mvc work fine local m...,|asp.net|asp.net-mvc|bundling-and-minification|,24395412,40,49177,0.0,12
1,ajax request made flux app,creating react application flux architecture t...,|javascript|reactjs|reactjs-flux|,26632415,199,40698,0.0,6
2,gem install debugger error,running rail mac everything smoothly sudden ge...,|ruby-on-rails|ruby|rubygems|rvm|,24395453,25,28278,0.0,6
3,change body parser limit firebase,trying proxy file upload firebase cloud functi...,|node.js|express|firebase|google-cloud-functio...,45489810,7,946,NaN,1
4,io android material design hierarchical timing...,want animation introduced android material des...,|ios|uicollectionview|calayer|material-design|,26632893,11,895,0.0,3
...,...,...,...,...,...,...,...,...
49995,make toolbar grid layout,want make qtoolbar column button docked left s...,|c++|qt|qt5|,23638595,7,2137,NaN,1
49996,using jmslistener multiple payload type destin...,write instance multiple type given destination...,|java|spring|jms|spring-jms|,42656519,6,6453,0.0,3
49997,testing resolved data test ngoninit,working testing guide wish write test ngoninit...,|javascript|angular|jasmine|karma-runner|resol...,42656045,34,67797,0.0,2
49998,unit test project reference mvc project,perhaps easier let visual studio create unit t...,|unit-testing|visual-studio-2013|asp.net-mvc-5|,23638602,10,7610,0.0,4


In [5]:
df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Title', 'Body', 'Tags', 'Id', 'Score', 'ViewCount', 'FavoriteCount',
       'AnswerCount'],
      dtype='object')

## **Feature engineering**

In [6]:
df.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Title              26
Body                0
Tags                0
Id                  0
Score               0
ViewCount           0
FavoriteCount    7645
AnswerCount         0
dtype: int64

In [7]:
df= df.dropna(subset=['Title'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
X = df[['Title', 'Body']]
y = df['Tags']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# Initialiser un CountVectorizer
vectorizer = CountVectorizer()

# Adopter (fit) du vectorizer sur les données d'entraînement
vectorizer = vectorizer.fit(X_train['Title'])

# Appliquer le (transformer) du vectorizer sur les données d'entraînement  de test
X_features = vectorizer.transform(X['Title'] + ' ' + X['Body'])
X_train_features = vectorizer.transform(X_train['Title'] + ' ' + X_train['Body'])
X_test_features = vectorizer.transform(X_test['Title'] + ' ' + X_test['Body'])

# Affichage des caractéristiques (features) générées
print("Nombre de features (termes) :", len(vectorizer.get_feature_names_out()))
print("Exemple de features :", vectorizer.get_feature_names_out()[:20])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Nombre de features (termes) : 16076
Exemple de features : ['aaa' 'aab' 'aac' 'aaf' 'aapt' 'aar' 'abac' 'abandon' 'abandoned' 'abap'
 'abaqus' 'abbreviated' 'abbreviation' 'abc' 'abcd' 'abcde' 'abfss' 'abi'
 'abifilters' 'ability']


In [10]:
pd.DataFrame(X_train_features.toarray())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,...,16066,16067,16068,16069,16070,16071,16072,16073,16074,16075
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


## **Modélisation avec LDA**

### **Mesure de la perplexité en fonction du nombrede topics**

In [11]:
for i in range(4,31,2):
    n_topics = i

    print("Fitting LatentDirichletAllocation with n_topics=%d" % (n_topics,))

    lda = LatentDirichletAllocation(n_components=n_topics,max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    t0 = time()
    lda.fit(X_train_features)

    # train_gamma = lda.transform(X_train_features)
    # train_perplexity = lda.perplexity(X_train_features)

    test_gamma = lda.transform(X_test_features)
    test_perplexity = lda.perplexity(X_test_features)

    print('sklearn preplexity: test=%.3f' %
          (test_perplexity))

    print("done in %0.3fs." % (time() - t0))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fitting LatentDirichletAllocation with n_topics=4
sklearn preplexity: test=1621.607
done in 79.798s.
Fitting LatentDirichletAllocation with n_topics=6
sklearn preplexity: test=1593.515
done in 84.457s.
Fitting LatentDirichletAllocation with n_topics=8
sklearn preplexity: test=1541.800
done in 87.530s.
Fitting LatentDirichletAllocation with n_topics=10
sklearn preplexity: test=1501.761
done in 91.025s.
Fitting LatentDirichletAllocation with n_topics=12
sklearn preplexity: test=1508.458
done in 98.118s.
Fitting LatentDirichletAllocation with n_topics=14
sklearn preplexity: test=1480.607
done in 102.620s.
Fitting LatentDirichletAllocation with n_topics=16
sklearn preplexity: test=1494.745
done in 97.714s.
Fitting LatentDirichletAllocation with n_topics=18
sklearn preplexity: test=1463.158
done in 103.654s.
Fitting LatentDirichletAllocation with n_topics=20
sklearn preplexity: test=1481.511
done in 108.725s.
Fitting LatentDirichletAllocation with n_topics=22
sklearn preplexity: test=1451.6

### **Evolution du score de cohérence en fonction du nombre de topics**

In [12]:
import gensim
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
#import pyLDAvis.sklearn
import matplotlib.pyplot as plt


df = df.dropna(subset=['Title', 'Body'])

# Prétraitement du texte : tokenization et création du corpus
def preprocess_text(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    return tokens

df['tokens'] = df['Title'].apply(preprocess_text) + df['Body'].apply(preprocess_text)
texts = df['tokens'].apply(lambda x: ' '.join(x))

# Création de la matrice de termes
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(texts)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Fonction pour calculer les valeurs de cohérence
def compute_coherence_values_sklearn(vectorizer, X, texts, start, limit, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
        model.fit(X)
        model_list.append(model)

        # Transformer le modèle scikit-learn en un modèle Gensim pour calculer la cohérence
        topics = model.components_
        feature_names = vectorizer.get_feature_names_out()
        topic_words = [[feature_names[i] for i in topic.argsort()[:-10 - 1:-1]] for topic in topics]

        # Utiliser le modèle de cohérence de Gensim
        coherence_model = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())

    return model_list, coherence_values

# Création du dictionnaire (id2word) pour le calcul de la cohérence
dictionary = Dictionary(df['tokens'])
model_list, coherence_values = compute_coherence_values_sklearn(vectorizer, X, df['tokens'], start=4, limit=31, step=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Affichage du graphique
plt.plot(range(4, 31, 2), coherence_values)
plt.xlabel("Nombre de topics")
plt.ylabel("Score de cohérence C_v")
plt.title("Évolution du score de cohérence en fonction du nombre de topics")
plt.show()

**Au regard de la courbe du score de cohérence et des scores de perplexité, n-topics= 24 nous semble le plus optimal. En effet, il présente le 2ème score de cohérence le plus élevè tout en ayant également la 2ème perplexité la moins élevée**

### **Entraînement du modèle LDA**

In [ ]:
# Créez et entraînez votre modèle LDA
import time
lda = LatentDirichletAllocation(learning_method='online', learning_offset=50.0, max_iter=10, n_components=24, random_state=42)
start_time = time.time()
lda.fit(X_train_features)
end_time = time.time()

### **Visualisation des topics**

In [ ]:
def display_topics(model, feature_names, no_top_words):
  """
    Affiche les mots les plus significatifs pour chaque topic d'un modèle de Latent Dirichlet Allocation (LDA).

    Cette fonction parcourt les topics du modèle LDA et affiche les mots les plus significatifs pour chaque topic,
    basés sur leurs poids.

    Paramètres :
    model : LatentDirichletAllocation
        Le modèle LDA entraîné.
    feature_names : array-like
        Les noms des caractéristiques (mots) extraits du vectoriseur utilisé pour entraîner le modèle LDA.
    no_top_words : int
        Le nombre de mots les plus significatifs à afficher pour chaque topic.

    Exemple :
    >>> no_top_words = 20
    >>> display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)
    Topic 0:
    word1 word2 word3 ...
    Topic 1:
    word1 word2 word3 ...
    """
  for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 20
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)

1.   Topic 0 : Développement logiciel et informatique
2.   Topic 1 : Développement et déploiement dans le cloud
3.   Topic 2 : Traitement d'images et vision par ordinateur
1.   Topic 3 : Gestion des erreurs et débogage
2.   Topic 4 : Traitement de données et manipulation de chaînes de caractères
1.   Topic 5 : Gestion de données et bases de données
1.   Topic 6 : Développement d'applications et programmation
1.   Topic 7 : Sécurité informatique et authentification
1.   Topic 8 : Développement web et frameworks
1.   Topic 9 : Gestion de contenu et développement de fonctionnalités
1.   Topic 10 : Tests automatisés et validation de logiciels
1.   Topic 11 : Développement d'applications mobiles
1.   Topic 12 : Programmation orientée objet et développement logiciel
2.   Topic 13 : Traitement de données et manipulation de fichiers
2.   Topic 14 : Interfaces utilisateur et conception graphique
2.   Topic 15 :Traitement audio et multimédia
2.   Topic 16 :Développement web et frameworks
2.   Topic 17 : Développement et programmation
2.   Topic 18 : Débogage et optimisation de code
2.   Topic 19 : Développement et déploiement dans le cloud
2.   Topic 20 : Développement web et interfaces utilisateur
1.   Topic 21 : Développement logiciel et programmation
1.   Topic 22 : Développement mobile et programmation
2.   Topic 23 : Gestion de données et bases de données










In [ ]:
stop_words = set(stopwords.words("english"))
num_rows = len(lda.components_) // 3 + 1
tf_feature_names =vectorizer.get_feature_names_out()
def wordcloud_lda(lda, tf_feature_names):
  """
    Génère et affiche des nuages de mots pour chaque topic d'un modèle de Latent Dirichlet Allocation (LDA).

    Cette fonction crée des nuages de mots pour les mots les plus significatifs de chaque topic d'un modèle LDA,
    et les affiche dans une grille de sous-graphes.

    Paramètres :
    lda : LatentDirichletAllocation
        Le modèle LDA entraîné.
    tf_feature_names : array-like
        Les noms des caractéristiques (mots) extraits du vectoriseur utilisé pour entraîner le modèle LDA.

    Exemple :
    >>> wc = wordcloud_lda(lda, tf_feature_names)
    """

  fig, axs = plt.subplots(len(lda.components_) // 3, 3, figsize=(10, 5*num_rows))

  for i in range(len(lda.components_)):
        corpus_lda = lda.components_[i]
        first_topic_words = [
            tf_feature_names[l] for l in corpus_lda.argsort()[: -50 - 1 : -1]
        ]
        k = i // 3
        j = i - k * 3
        wordcloud = WordCloud(
            stopwords=stop_words, background_color="black", width=2500, height=1800
        )
        wordcloud = wordcloud.generate(" ".join(first_topic_words))
        axs[k][j].set_title("Wordcloud pour le \nsujet {}".format(i))
        axs[k][j].axis("off")
        axs[k][j].imshow(wordcloud)

  r = len(lda.components_) % 3
  [
        fig.delaxes(axs[len(lda.components_) // 3, k - 1])
        for k in range(r + 1, 3 + 1)
        if r != 0
  ]


wc = wordcloud_lda(lda, tf_feature_names)
wc

In [ ]:
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis

In [ ]:
# Appliquer le prétraitement au DataFrame
df['tokens'] = df['Title'].apply(preprocess_text) + df['Body'].apply(preprocess_text)

# Création du dictionnaire (id2word) et du corpus de documents
dictionary = Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in df['tokens']]

# Entraînement d'un modèle LDA
lda_model = LdaModel(corpus, num_topics=24, id2word=dictionary, passes=20)

# Visualisation des topics avec pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'lda_visualization.html')  # Sauvegarder la visualisation au format HTML
pyLDAvis.display(vis)  # Afficher la visualisation dans le notebook

### **Taux de couverture des tags réels**

In [ ]:
# Convertir les tags en une liste de listes
y_true = y_test.str.split().tolist()

In [ ]:
# Utilisez le modèle pour prédire les mots-clés pour les questions de test
y_pred = lda.transform(X_test_features)

In [ ]:
# Convertir les prédictions en un format approprié
y_pred_labels = [max(enumerate(pred), key=lambda x: x[1])[0] for pred in y_pred]

# Convertir les étiquettes réelles en un format approprié
true_labels = [max(enumerate(true), key=lambda x: x[1])[0] for true in y_true]

# Calculer les métriques
f1 = f1_score(true_labels, y_pred_labels, average='weighted')
accuracy = accuracy_score(true_labels, y_pred_labels)
jaccard = jaccard_score(true_labels, y_pred_labels, average='weighted')
#recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(true_labels, y_pred_labels, average='weighted')

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
#("recall", round(recall, 2))
print(f"Jaccard Score: {jaccard}")
print(f"Precision: {precision}")
print("training_time", end_time - start_time)

### **Taux de couverture des mots des questions**

In [ ]:
# Utilisez le modèle pour prédire les mots-clés pour les questions de test
y_pred = lda.transform(X_test_features)

# Calculer la diversité des mots-clés proposés
average_word_coverage = np.mean(np.sum(y_pred > 0, axis=1))

print("Average word coverage:", average_word_coverage)


### **Test de visu sur quelques questions**

In [ ]:
# Exemple d'évaluation visuelle sur les 10 premières questions
for i in range(5):
    print("Question:", i)
    print("Tags réels:", y_true[i])
    print("Mots-clés prédits:", y_pred[i])
    print("\n")
